In [1]:
import plotly
import pandas as pd
from pathlib import Path
import plotly.express as px
import plotly.io as io
import os

In [74]:
df = pd.read_csv("cov/result_PVAL_71_S7.hist", 
                 sep="\t", 
                 names=["chrom", "start", "end", "name", "score", "strand","depth","num_bases_at_depth","size_of_feature","pros_of_feature_at_depth"])
df = df[df.chrom != "all"].copy()
df[['ID', 'rest']] = df['name'].str.split('_cds_', -1, expand=True) # https://stackoverflow.com/a/39358924
df[["exon_number", "unknown", "exon_chrom", "exon_start_pos", "exon_strand"]] = df['rest'].str.split('_', -1, expand=True)
df = df.drop(["name","rest"], axis = 1)

df = df.astype({
           'chrom':'str',
           'start':'int',
           'end':'int',
           'score':'float',
           'strand':'str',
           'depth':'int',
           'num_bases_at_depth':'int',
           'size_of_feature':'int',
           'pros_of_feature_at_depth':'float',
           'ID':'category',
           "exon_number":'category', 
           "unknown":'category', 
           "exon_chrom":"category", 
           "exon_start_pos":"int", 
           "exon_strand":"category"
          }
         )
transcripts = df['ID'].unique()

grouped = df.groupby(df.ID)

transcripts_list = []

for isof in transcripts:
    c = grouped.get_group(isof)
    # Scrape NCBI transcript ID
    NCBI_id = c.iloc[0]['ID']
    # Duplicate rows with several bases 
    c = c.loc[c.index.repeat(c.num_bases_at_depth)] # https://stackoverflow.com/a/57009491
    # Extract key values for the depth column
    c = c.describe()['depth'].to_frame(NCBI_id).T
    transcripts_list.append(c)

# Join all key data into one df
all_transcripts = (pd.concat(transcripts_list, axis=0)
                   .rename(columns={'count': 'total_exons_length'})
                   .astype({
                           'total_exons_length':'int',
                           'max':'int',
                           'min':'int'}
                          )                  
                  )
all_transcripts.to_csv()


In [10]:
gene_names = pd.read_csv("gene_names.tsv", 
                     sep="\t", 
                     names=["name", "symbol"])
gene_names

,name,symbol
0,name,name2
1,NM_000110,DPYD
2,NM_001007792,NTRK1
3,NM_001012331,NTRK1
4,NM_001014796,DDR2
...,...,...
404,NM_001349956,CHEK2
405,NM_001362877,SMARCB1
406,NM_003073,SMARCB1
407,NM_007194,CHEK2


In [15]:
import plotly
import pandas as pd
from pathlib import Path
import plotly.express as px
import plotly.io as io
import os

sample_covs = Path('all_samples_joined.tsv')


gene_names = pd.read_csv("gene_names.tsv", 
                         sep="\t", 
                         names=["name", "symbol"],
                         header=0
                        )

header = ["sample_id", "chrom", "start", "end", "name", "score", "strand","depth","num_bases_at_depth","size_of_feature","pros_of_feature_at_depth"]

all_samples = pd.read_csv(sample_covs, 
                          sep="\t", 
                          names=header)

fig_list = []
metrics_df_list = []
out_path = "sample_coverages/"

grouped = all_samples.groupby(all_samples.sample_id)


sample_IDs = all_samples['sample_id'].unique()

transcripts_list = []



for sample in sample_IDs:
    df = grouped.get_group(sample)
    df = df[df.chrom != "all"].copy()
    df = df[df.chrom != "all"].copy()
    df[['ID', 'rest']] = df['name'].str.split('_cds_', -1, expand=True) # https://stackoverflow.com/a/39358924
    df[["exon_number", "unknown", "exon_chrom", "exon_start_pos", "exon_strand"]] = df['rest'].str.split('_', -1, expand=True)
    df[['base_ID', 'version']] = df['ID'].str.split('.', 2, expand=True)
    sample_name = df.iloc[0][header[0]]
    df = df.drop(["name","rest"]+[header[0]], axis = 1)
    df = df.astype({
           'chrom':'str',
           'start':'int',
           'end':'int',
           'score':'float',
           'strand':'str',
           'depth':'int',
           'num_bases_at_depth':'int',
           'size_of_feature':'int',
           'pros_of_feature_at_depth':'float',
           'ID':'category',
           "exon_number":'category', 
           "unknown":'category', 
           "exon_chrom":"category", 
           "exon_start_pos":"int", 
           "exon_strand":"category"
          }
         )
    df = df.merge(right=gene_names,
                  how='left',
                  left_on='base_ID',
                  right_on='name'
                 )
    # Join symbols and IDs into one column so they can be visualised as one entity
    df['symbol_ID'] = df['symbol'] + " " + df['ID'].astype(str)
    # Sort the symbol_IDs so they aren't randomly presented in the figure
    df = df.sort_values(by=['symbol_ID'])
    # Duplicate rows with several bases, this enables calculating averages in a more easier way
    df = df.loc[df.index.repeat(df.num_bases_at_depth)] # https://stackoverflow.com/a/57009491
    
    # Get a list of symbol_IDs so they can be looped through later on
    transcripts = df['symbol_ID'].unique()
    
    grouped = df.groupby(df.symbol_ID)

    transcripts_list = []
    
    for i in transcripts:
        c = grouped.get_group(i)
        # Scrape NCBI transcript ID
        NCBI_id = c.iloc[0]['symbol_ID']
        
        # Extract key values for the depth column
        c = c.describe()['depth'].to_frame(NCBI_id).T
        transcripts_list.append(c)

    # Join all key data into one df
    all_transcripts = (pd.concat(transcripts_list, axis=0)
                       .rename(columns={'count': 'total_length_of_exons'})
                       .astype({
                               'total_length_of_exons':'int',
                               'max':'int',
                               'min':'int'}
                              )                  
                      )
    
    all_transcripts.index.name = "ID"
    # Create bar plots
    bar_fig = px.bar(all_transcripts.reset_index(), 
                     y='mean', 
                     x='ID', 
                     hover_data=["total_length_of_exons", "mean", 'std', 'min', '25%', '50%', '75%', 'max',"ID"],
                     title="Sample name: " + sample_name
                    )
    #bar_fig.write_html(out_path + "bar/" + sample_name + "_bar" + ".html", 
    #                   config= {'displaylogo': False})
    
    fig_list.append(bar_fig)
    
    # Create tables with metrics
    all_transcripts['sample_name'] = sample_name
    metrics_df_list.append(all_transcripts)

,chrom,start,end,score,strand,depth,num_bases_at_depth,size_of_feature,pros_of_feature_at_depth,ID,exon_number,unknown,exon_chrom,exon_start_pos,exon_strand,base_ID,version,name,symbol,symbol_ID
312918,chr14,105241412,105241544,0.0,-,628,2,132,0.015152,NM_001014431.2,8,0,chr14,105241413,r,NM_001014431,2,NM_001014431,AKT1,AKT1 NM_001014431.2
312918,chr14,105241412,105241544,0.0,-,628,2,132,0.015152,NM_001014431.2,8,0,chr14,105241413,r,NM_001014431,2,NM_001014431,AKT1,AKT1 NM_001014431.2
312945,chr14,105241412,105241544,0.0,-,692,2,132,0.015152,NM_001014431.2,8,0,chr14,105241413,r,NM_001014431,2,NM_001014431,AKT1,AKT1 NM_001014431.2
312945,chr14,105241412,105241544,0.0,-,692,2,132,0.015152,NM_001014431.2,8,0,chr14,105241413,r,NM_001014431,2,NM_001014431,AKT1,AKT1 NM_001014431.2
312946,chr14,105241412,105241544,0.0,-,693,1,132,0.007576,NM_001014431.2,8,0,chr14,105241413,r,NM_001014431,2,NM_001014431,AKT1,AKT1 NM_001014431.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26954,chr2,234668933,234669797,0.0,+,663,4,864,0.004630,NM_000463.3,0,0,chr2,234668934,f,NM_000463,3,NM_000463,UGT1A1,UGT1A1 NM_000463.3
26955,chr2,234668933,234669797,0.0,+,664,1,864,0.001157,NM_000463.3,0,0,chr2,234668934,f,NM_000463,3,NM_000463,UGT1A1,UGT1A1 NM_000463.3
27377,chr2,234680907,234681205,0.0,+,628,1,298,0.003356,NM_000463.3,4,0,chr2,234680908,f,NM_000463,3,NM_000463,UGT1A1,UGT1A1 NM_000463.3
26745,chr2,234668933,234669797,0.0,+,351,2,864,0.002315,NM_000463.3,0,0,chr2,234668934,f,NM_000463,3,NM_000463,UGT1A1,UGT1A1 NM_000463.3


In [12]:
all_metrics = pd.concat(metrics_df_list, axis=0, ignore_index=True)
out_path = Path(out_path + "metrics/all_metrics.csv")
all_metrics.to_csv(out_path)

In [13]:
all_bar_plots = 'sample_coverages/bar/bar_plot_all_samples.html'
# Remove the previous version of html file
if os.path.exists(all_bar_plots):
    os.remove(all_bar_plots)

# https://stackoverflow.com/a/59869358
# Write all bar plots into one html page
with open(all_bar_plots, 'a') as f:
    for fig in fig_list:
        f.write(fig.to_html(full_html=False, 
                            include_plotlyjs='cdn', 
                            config= {'displaylogo': False}))

In [39]:
all_transcripts.columns

Index(['total_exons_length', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'], dtype='object')

In [33]:
transcripts

array(['DPYD_NM_000110.4', 'NTRK1_NM_001007792.1', 'NTRK1_NM_001012331.2',
       'DDR2_NM_001014796.3', 'MUTYH_NM_001048171.2',
       'MUTYH_NM_001048172.2', 'MUTYH_NM_001048173.2',
       'MUTYH_NM_001048174.2', 'MUTYH_NM_001128425.2',
       'DPYD_NM_001160301.1', 'MUTYH_NM_001293190.2',
       'MUTYH_NM_001293191.2', 'MUTYH_NM_001293192.2',
       'MUTYH_NM_001293195.2', 'MUTYH_NM_001293196.2',
       'MUTYH_NM_001350650.2', 'MUTYH_NM_001350651.2',
       'DDR2_NM_001354982.2', 'DDR2_NM_001354983.2', 'NRAS_NM_002524.5',
       'NTRK1_NM_002529.4', 'DDR2_NM_006182.4', 'MUTYH_NM_012222.3',
       'MSH6_NM_000179.3', 'MSH2_NM_000251.3', 'UGT1A1_NM_000463.3',
       'MSH2_NM_001258281.1', 'MSH6_NM_001281492.2',
       'MSH6_NM_001281493.2', 'MSH6_NM_001281494.2',
       'IDH1_NM_001282386.1', 'IDH1_NM_001282387.1',
       'MYCN_NM_001293228.2', 'MYCN_NM_001293231.2',
       'MYCN_NM_001293233.2', 'ALK_NM_001353765.2', 'ALK_NM_004304.5',
       'MYCN_NM_005378.6', 'IDH1_NM_005896.4', '

In [34]:
df

,chrom,start,end,score,strand,depth,num_bases_at_depth,size_of_feature,pros_of_feature_at_depth,ID,exon_number,unknown,exon_chrom,exon_start_pos,exon_strand,base_ID,version,name,symbol,symbol_ID
0,chr1,97544531,97544702,0.0,-,125,3,171,0.017544,NM_000110.4,0,0,chr1,97544532,r,NM_000110,4,NM_000110,DPYD,DPYD_NM_000110.4
0,chr1,97544531,97544702,0.0,-,125,3,171,0.017544,NM_000110.4,0,0,chr1,97544532,r,NM_000110,4,NM_000110,DPYD,DPYD_NM_000110.4
0,chr1,97544531,97544702,0.0,-,125,3,171,0.017544,NM_000110.4,0,0,chr1,97544532,r,NM_000110,4,NM_000110,DPYD,DPYD_NM_000110.4
1,chr1,97544531,97544702,0.0,-,126,1,171,0.005848,NM_000110.4,0,0,chr1,97544532,r,NM_000110,4,NM_000110,DPYD,DPYD_NM_000110.4
2,chr1,97544531,97544702,0.0,-,128,2,171,0.011696,NM_000110.4,0,0,chr1,97544532,r,NM_000110,4,NM_000110,DPYD,DPYD_NM_000110.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482805,chr22,29130390,29130709,0.0,-,589,1,319,0.003135,NM_145862.2,12,0,chr22,29130391,r,NM_145862,2,NM_145862,CHEK2,CHEK2_NM_145862.2
482806,chr22,29130390,29130709,0.0,-,590,1,319,0.003135,NM_145862.2,12,0,chr22,29130391,r,NM_145862,2,NM_145862,CHEK2,CHEK2_NM_145862.2
482807,chr22,29130390,29130709,0.0,-,593,3,319,0.009404,NM_145862.2,12,0,chr22,29130391,r,NM_145862,2,NM_145862,CHEK2,CHEK2_NM_145862.2
482807,chr22,29130390,29130709,0.0,-,593,3,319,0.009404,NM_145862.2,12,0,chr22,29130391,r,NM_145862,2,NM_145862,CHEK2,CHEK2_NM_145862.2
